In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from scipy.stats import norm, gaussian_kde
import matplotlib.pyplot as plt

In [2]:
MISS_NUM = '24' # 6, 12, 18, 24
DATA_NUM = '9918' # 9918

In [3]:
Data = pd.read_csv("Data"+DATA_NUM+"_30min.csv", index_col=0)
num_day  = 366
num_time = 48
data = np.array(Data)/np.max(np.array(Data))

In [4]:
range1 = [6*x-6 for x in range(1,62)]
range2 = [6*x-5 for x in range(1,62)]
range3 = [6*x-4 for x in range(1,62)]
range4 = [6*x-3 for x in range(1,62)]
range5 = [6*x-2 for x in range(1,62)]
range6 = [6*x-1 for x in range(1,62)]

range_train = range1 + range2 + range3 + range4
range_val   = range5
range_test  = range6

l  = int(MISS_NUM)
l2 = int(l/2)

mean_corrector_train = 48/l2
mean_corrector_val  = 48/l2
mean_corrector_test  = 48/l

try:
    data_miss = np.array(pd.read_csv("DATA"+DATA_NUM+"_MISS"+MISS_NUM+".csv", index_col=0))
except:
    data_miss = data.copy()
    for i in range(num_day):
        if i in range_test:
            s = np.random.choice(range(1, 48-l))
            data_miss[i][s:s+l] = 0
        else:
            s = np.random.choice(range(1, 48-l2))
            data_miss[i][s:s+l2] = 0

    pd.DataFrame(data_miss).to_csv("DATA"+DATA_NUM+"_MISS"+MISS_NUM+".csv")

In [5]:
data_train_miss  = np.zeros((1,48))
data_train_miss2 = np.zeros((1,48))
data_val_miss    = np.zeros((1,48))
data_val_miss2   = np.zeros((1,48))
for i in range(len(data_miss)):
    j = np.where(data_miss[i]==0)[0][0]
    k = np.where(data_miss[i]==0)[0][-1]

    if j > l2:
        data_miss_sample = data_miss[i:i+1].copy()
        data_miss_sample[0][j-l2:j] = 0
        if i in range_train:
            data_train_miss  = np.append(data_train_miss, data_miss[i:i+1], axis=0)
            data_train_miss2 = np.append(data_train_miss2, data_miss_sample, axis=0)
        if i in range_val:
            data_val_miss  = np.append(data_val_miss, data_miss[i:i+1], axis=0)
            data_val_miss2 = np.append(data_val_miss2, data_miss_sample, axis=0)
    if k < 47-l2:
        data_miss_sample = data_miss[i:i+1].copy()
        data_miss_sample[0][k+1:k+1+l2] = 0
        if i in range_train:
            data_train_miss  = np.append(data_train_miss, data_miss[i:i+1], axis=0)
            data_train_miss2 = np.append(data_train_miss2, data_miss_sample, axis=0)
        if i in range_val:
            data_val_miss  = np.append(data_val_miss, data_miss[i:i+1], axis=0)
            data_val_miss2 = np.append(data_val_miss2, data_miss_sample, axis=0)

data_train_miss  = data_train_miss[1:]
data_train_miss2 = data_train_miss2[1:]
data_val_miss    = data_val_miss[1:]
data_val_miss2   = data_val_miss2[1:]
data_test        = data[range_test]
data_test_miss   = data_miss[range_test]

In [ ]:
# Historical Average

data_test_HA = data_test_miss.copy()
for i in range(len(data_test_HA)):
    j = np.where(data_test_miss[i]==0)[0][0]
    d1 = max(range_test[i % len(range_test)] - 1, 0)
    d2 = min(range_test[i % len(range_test)] + 1, len(data)-1)
    data_test_HA[i][j:j+l] = 0.5*(data[d1][j:j+l] + data[d2][j:j+l])
data_test_HA[data_test_miss != 0] = 0

mse_test = np.mean(np.square(data_test_HA - (data_test - data_test_miss)))*mean_corrector_test
mae_test = np.mean(np.abs(data_test_HA - (data_test - data_test_miss)))*mean_corrector_test

NRMSE_test = round(100*np.sqrt(mse_test),2)
NMAE_test  = round(100*mae_test,2)

print("NRMSE_test: {}%".format(NRMSE_test).ljust(25), end="")
print("NMAE_test: {}%".format(NMAE_test).ljust(25))

In [ ]:
# Linear Interpolation

def LI(v_left, v_right):
    x = np.array(range(1,l+1))
    y = (v_right - v_left)*x/(l+1) + v_left
    return y

data_test_LI = data_test_miss.copy()
for i in range(len(data_test_LI)):
    j = np.where(data_test_miss[i]==0)[0][0]
    j_left  = np.where(data_test_miss[i]==0)[0][0] - 1
    j_right = np.where(data_test_miss[i]==0)[0][-1] + 1
    v_left  = data_test_miss[i][j_left]
    v_right = data_test_miss[i][j_right]
    data_test_LI[i][j:j+l] = LI(v_left, v_right)
data_test_LI[data_test_miss != 0] = 0

mse_test = np.mean(np.square(data_test_LI - (data_test - data_test_miss)))*mean_corrector_test
mae_test = np.mean(np.abs(data_test_LI - (data_test - data_test_miss)))*mean_corrector_test

NRMSE_test = round(100*np.sqrt(mse_test),2)
NMAE_test  = round(100*mae_test,2)

print("NRMSE_test: {}%".format(NRMSE_test).ljust(25), end="")
print("NMAE_test: {}%".format(NMAE_test).ljust(25))

In [8]:
# AE

class AE(nn.Module):
    def __init__(self):
        super(AE, self).__init__()
        self.fc1 = nn.Linear(48, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 32)
        self.fc4 = nn.Linear(32, 48)
        self.activation = nn.Sigmoid() #nn.ReLU()

    def forward_train(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        z = self.fc2(x)
        y = self.activation(z)
        y = self.fc3(y)
        y = self.activation(y)
        y = self.fc4(y)
        return z, y

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        z = self.fc2(x)
        y = self.activation(z)
        y = self.fc3(y)
        y = self.activation(y)
        y = self.fc4(y)
        return z, y

class AED(nn.Module):
    def __init__(self):
        super(AED, self).__init__()
        self.fc1 = nn.Linear(48, 32)
        self.dropout1 = nn.Dropout(0.1)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 32)
        self.dropout2 = nn.Dropout(0.1)
        self.fc4 = nn.Linear(32, 48)
        self.activation = nn.Sigmoid() #nn.ReLU()

    def forward_train(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.dropout1(x)
        z = self.fc2(x)
        y = self.activation(z)
        y = self.fc3(y)
        y = self.activation(y)
        y = self.dropout2(y)
        y = self.fc4(y)
        return z, y

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        z = self.fc2(x)
        y = self.activation(z)
        y = self.fc3(y)
        y = self.activation(y)
        y = self.fc4(y)
        return z, y

def train_AE(model, batch, optimizer):
    x, y = batch[0], batch[1]
    x_hat = model.forward_train(x)[1]

    loss = nn.functional.mse_loss(x_hat, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

def train_MAE(model, batch, optimizer):
    x, y = batch[0], batch[1]
    x_hat = model.forward_train(x)[1]
    x_hat[y == 0] = 0

    loss = nn.functional.mse_loss(x_hat, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
# Denoising AutoEncoder (Training)

batch_size1 = 128
total_batch1 = int(len(data_train_miss)/batch_size1) + 1
batch_size2 = 32
total_batch2 = int(len(data_val_miss)/batch_size2) + 1

dae = AED()
optimizer = optim.Adam(dae.parameters(), lr = 0.001)

mse_train = []; mse_val = []; mse_test = []
mae_train = []; mae_val = []; mae_test = []
for epoch in range(10000):
    for i in range(total_batch1):
        batch_x = torch.tensor(data_train_miss2[batch_size1*i:batch_size1*(i+1),:] ,dtype=torch.float)
        batch_y = torch.tensor(data_train_miss[batch_size1*i:batch_size1*(i+1),:] ,dtype=torch.float)
        batch = [batch_x, batch_y]
        train_AE(dae, batch, optimizer)

    data_train_DAE = dae.forward(torch.tensor(data_train_miss2, dtype=torch.float))[1].detach().numpy()
    data_val_DAE   = dae.forward(torch.tensor(data_val_miss2, dtype=torch.float))[1].detach().numpy()
    data_test_DAE  = dae.forward(torch.tensor(data_test_miss, dtype=torch.float))[1].detach().numpy()

    data_train_DAE[data_train_miss - data_train_miss2 == 0] = 0
    data_val_DAE[data_val_miss - data_val_miss2 == 0] = 0
    data_test_DAE[data_test_miss != 0] = 0

    mse_train += [np.mean(np.square(data_train_DAE - (data_train_miss - data_train_miss2)))*mean_corrector_train]
    mae_train += [np.mean(np.abs(data_train_DAE - (data_train_miss - data_train_miss2)))*mean_corrector_train]
    mse_val += [np.mean(np.square(data_val_DAE - (data_val_miss - data_val_miss2)))*mean_corrector_val]
    mae_val += [np.mean(np.abs(data_val_DAE - (data_val_miss - data_val_miss2)))*mean_corrector_val]
    mse_test += [np.mean(np.square(data_test_DAE - (data_test - data_test_miss)))*mean_corrector_test]
    mae_test += [np.mean(np.abs(data_test_DAE - (data_test - data_test_miss)))*mean_corrector_test]

    NRMSE_train = round(100*np.sqrt(mse_train[-1]),2)
    NMAE_train  = round(100*mae_train[-1],2)
    NRMSE_val = round(100*np.sqrt(mse_val[-1]),2)
    NMAE_val  = round(100*mae_val[-1],2)
    NRMSE_test = round(100*np.sqrt(mse_test[-1]),2)
    NMAE_test  = round(100*mae_test[-1],2)

    if mse_val[-1] == np.min(mse_val):
        torch.save(dae.state_dict(), 'DAE_'+DATA_NUM+'_'+MISS_NUM+'.pt')

    if epoch == 0 or (epoch+1) % 100 == 0:
        print("epoch: {}".format(epoch+1).ljust(15), end="")
        print("NRMSE_train: {}%".format(NRMSE_train).ljust(25), end="")
        print("NMAE_train: {}%".format(NMAE_train).ljust(25), end="")
        print("NRMSE_val: {}%".format(NRMSE_val).ljust(25), end="")
        print("NMAE_val: {}%".format(NMAE_val).ljust(25), end="")
        print("NRMSE_test: {}%".format(NRMSE_test).ljust(25), end="")
        print("NMAE_test: {}%".format(NMAE_test).ljust(25))

In [ ]:
# Denoising AutoEncoder (Test)

batch_size = 128
total_batch = int(len(data_train_miss)/batch_size) + 1

dae = AED()

mse_train = []; mse_val = []; mse_test = []
mae_train = []; mae_val = []; mae_test = []

dae.load_state_dict(torch.load('DAE_'+DATA_NUM+'_'+MISS_NUM+'.pt'))

data_train_DAE = dae.forward(torch.tensor(data_train_miss2, dtype=torch.float))[1].detach().numpy()
data_val_DAE   = dae.forward(torch.tensor(data_val_miss2, dtype=torch.float))[1].detach().numpy()
data_test_DAE  = dae.forward(torch.tensor(data_test_miss, dtype=torch.float))[1].detach().numpy()

data_train_DAE[data_train_miss - data_train_miss2 == 0] = 0
data_val_DAE[data_val_miss - data_val_miss2 == 0] = 0
data_test_DAE[data_test_miss != 0] = 0

mse_train += [np.mean(np.square(data_train_DAE - (data_train_miss - data_train_miss2)))*mean_corrector_train]
mae_train += [np.mean(np.abs(data_train_DAE - (data_train_miss - data_train_miss2)))*mean_corrector_train]
mse_val += [np.mean(np.square(data_val_DAE - (data_val_miss - data_val_miss2)))*mean_corrector_val]
mae_val += [np.mean(np.abs(data_val_DAE - (data_val_miss - data_val_miss2)))*mean_corrector_val]
mse_test += [np.mean(np.square(data_test_DAE - (data_test - data_test_miss)))*mean_corrector_test]
mae_test += [np.mean(np.abs(data_test_DAE - (data_test - data_test_miss)))*mean_corrector_test]

NRMSE_train = round(100*np.sqrt(mse_train[-1]),2)
NMAE_train  = round(100*mae_train[-1],2)
NRMSE_val = round(100*np.sqrt(mse_val[-1]),2)
NMAE_val  = round(100*mae_val[-1],2)
NRMSE_test = round(100*np.sqrt(mse_test[-1]),2)
NMAE_test  = round(100*mae_test[-1],2)

print("NRMSE_train: {}%".format(NRMSE_train).ljust(25), end="")
print("NMAE_train: {}%".format(NMAE_train).ljust(25), end="")
print("NRMSE_val: {}%".format(NRMSE_val).ljust(25), end="")
print("NMAE_val: {}%".format(NMAE_val).ljust(25), end="")
print("NRMSE_test: {}%".format(NRMSE_test).ljust(25), end="")
print("NMAE_test: {}%".format(NMAE_test).ljust(25))

In [ ]:
# Masked AutoEncoder (Training)

batch_size1 = 128
total_batch1 = int(len(data_train_miss)/batch_size1) + 1
batch_size2 = 32
total_batch2 = int(len(data_val_miss)/batch_size2) + 1

mae = AED()
optimizer = optim.Adam(mae.parameters(), lr = 0.001)

mse_train = []; mse_val = []; mse_test = []
mae_train = []; mae_val = []; mae_test = []
for epoch in range(10000):
    for i in range(total_batch1):
        batch_x = torch.tensor(data_test_miss[batch_size1*i:batch_size1*(i+1),:] ,dtype=torch.float)
        batch_y = torch.tensor(data_test_miss[batch_size1*i:batch_size1*(i+1),:] ,dtype=torch.float)
        batch = [batch_x, batch_y]
        train_MAE(mae, batch, optimizer)

    data_train_MAE = mae.forward(torch.tensor(data_train_miss2, dtype=torch.float))[1].detach().numpy()
    data_val_MAE   = mae.forward(torch.tensor(data_val_miss2, dtype=torch.float))[1].detach().numpy()
    data_test_MAE  = mae.forward(torch.tensor(data_test_miss, dtype=torch.float))[1].detach().numpy()

    data_train_MAE[data_train_miss - data_train_miss2 == 0] = 0
    data_val_MAE[data_val_miss - data_val_miss2 == 0] = 0
    data_test_MAE[data_test_miss != 0] = 0

    mse_train += [np.mean(np.square(data_train_MAE - (data_train_miss - data_train_miss2)))*mean_corrector_train]
    mae_train += [np.mean(np.abs(data_train_MAE - (data_train_miss - data_train_miss2)))*mean_corrector_train]
    mse_val += [np.mean(np.square(data_val_MAE - (data_val_miss - data_val_miss2)))*mean_corrector_val]
    mae_val += [np.mean(np.abs(data_val_MAE - (data_val_miss - data_val_miss2)))*mean_corrector_val]
    mse_test += [np.mean(np.square(data_test_MAE - (data_test - data_test_miss)))*mean_corrector_test]
    mae_test += [np.mean(np.abs(data_test_MAE - (data_test - data_test_miss)))*mean_corrector_test]

    NRMSE_train = round(100*np.sqrt(mse_train[-1]),2)
    NMAE_train  = round(100*mae_train[-1],2)
    NRMSE_val = round(100*np.sqrt(mse_val[-1]),2)
    NMAE_val  = round(100*mae_val[-1],2)
    NRMSE_test = round(100*np.sqrt(mse_test[-1]),2)
    NMAE_test  = round(100*mae_test[-1],2)

    if mse_val[-1] == np.min(mse_val):
        torch.save(mae.state_dict(), 'MAE_'+DATA_NUM+'_'+MISS_NUM+'.pt')

    if epoch == 0 or (epoch+1) % 100 == 0:
        print("epoch: {}".format(epoch+1).ljust(15), end="")
        print("NRMSE_train: {}%".format(NRMSE_train).ljust(25), end="")
        print("NMAE_train: {}%".format(NMAE_train).ljust(25), end="")
        print("NRMSE_val: {}%".format(NRMSE_val).ljust(25), end="")
        print("NMAE_val: {}%".format(NMAE_val).ljust(25), end="")
        print("NRMSE_test: {}%".format(NRMSE_test).ljust(25), end="")
        print("NMAE_test: {}%".format(NMAE_test).ljust(25))

In [ ]:
# Masked AutoEncoder (Test)

batch_size = 128
total_batch = int(len(data_train_miss)/batch_size) + 1

mae = AED()

mse_train = []; mse_val = []; mse_test = []
mae_train = []; mae_val = []; mae_test = []

mae.load_state_dict(torch.load('MAE_'+DATA_NUM+'_'+MISS_NUM+'.pt'))

data_train_MAE = mae.forward(torch.tensor(data_train_miss2, dtype=torch.float))[1].detach().numpy()
data_val_MAE   = mae.forward(torch.tensor(data_val_miss2, dtype=torch.float))[1].detach().numpy()
data_test_MAE  = mae.forward(torch.tensor(data_test_miss, dtype=torch.float))[1].detach().numpy()

data_train_MAE[data_train_miss - data_train_miss2 == 0] = 0
data_val_MAE[data_val_miss - data_val_miss2 == 0] = 0
data_test_MAE[data_test_miss != 0] = 0

mse_train += [np.mean(np.square(data_train_MAE - (data_train_miss - data_train_miss2)))*mean_corrector_train]
mae_train += [np.mean(np.abs(data_train_MAE - (data_train_miss - data_train_miss2)))*mean_corrector_train]
mse_val += [np.mean(np.square(data_val_MAE - (data_val_miss - data_val_miss2)))*mean_corrector_val]
mae_val += [np.mean(np.abs(data_val_MAE - (data_val_miss - data_val_miss2)))*mean_corrector_val]
mse_test += [np.mean(np.square(data_test_MAE - (data_test - data_test_miss)))*mean_corrector_test]
mae_test += [np.mean(np.abs(data_test_MAE - (data_test - data_test_miss)))*mean_corrector_test]

NRMSE_train = round(100*np.sqrt(mse_train[-1]),2)
NMAE_train  = round(100*mae_train[-1],2)
NRMSE_val = round(100*np.sqrt(mse_val[-1]),2)
NMAE_val  = round(100*mae_val[-1],2)
NRMSE_test = round(100*np.sqrt(mse_test[-1]),2)
NMAE_test  = round(100*mae_test[-1],2)

print("NRMSE_train: {}%".format(NRMSE_train).ljust(25), end="")
print("NMAE_train: {}%".format(NMAE_train).ljust(25), end="")
print("NRMSE_val: {}%".format(NRMSE_val).ljust(25), end="")
print("NMAE_val: {}%".format(NMAE_val).ljust(25), end="")
print("NRMSE_test: {}%".format(NRMSE_test).ljust(25), end="")
print("NMAE_test: {}%".format(NMAE_test).ljust(25))

In [ ]:
# Denoising Masked AutoEncoder 1 (Training)

batch_size1 = 128
total_batch1 = int(len(data_train_miss)/batch_size1) + 1
batch_size2 = 32
total_batch2 = int(len(data_val_miss)/batch_size2) + 1

dmae1 = AED()
optimizer = optim.Adam(dmae1.parameters(), lr = 0.001)

mse_train = []; mse_val = []; mse_test = []
mae_train = []; mae_val = []; mae_test = []
for epoch in range(20000):
    for i in range(total_batch1):
        batch_x = torch.tensor(data_train_miss2[batch_size1*i:batch_size1*(i+1),:] ,dtype=torch.float)
        batch_y = torch.tensor(data_train_miss[batch_size1*i:batch_size1*(i+1),:] ,dtype=torch.float)
        batch = [batch_x, batch_y]
        train_MAE(dmae1, batch, optimizer)

    data_train_DMAE1 = dmae1.forward(torch.tensor(data_train_miss2, dtype=torch.float))[1].detach().numpy()
    data_val_DMAE1   = dmae1.forward(torch.tensor(data_val_miss2, dtype=torch.float))[1].detach().numpy()
    data_test_DMAE1  = dmae1.forward(torch.tensor(data_test_miss, dtype=torch.float))[1].detach().numpy()

    data_train_DMAE1[data_train_miss - data_train_miss2 == 0] = 0
    data_val_DMAE1[data_val_miss - data_val_miss2 == 0] = 0
    data_test_DMAE1[data_test_miss != 0] = 0

    mse_train += [np.mean(np.square(data_train_DMAE1 - (data_train_miss - data_train_miss2)))*mean_corrector_train]
    mae_train += [np.mean(np.abs(data_train_DMAE1 - (data_train_miss - data_train_miss2)))*mean_corrector_train]
    mse_val += [np.mean(np.square(data_val_DMAE1 - (data_val_miss - data_val_miss2)))*mean_corrector_val]
    mae_val += [np.mean(np.abs(data_val_DMAE1 - (data_val_miss - data_val_miss2)))*mean_corrector_val]
    mse_test += [np.mean(np.square(data_test_DMAE1 - (data_test - data_test_miss)))*mean_corrector_test]
    mae_test += [np.mean(np.abs(data_test_DMAE1 - (data_test - data_test_miss)))*mean_corrector_test]

    NRMSE_train = round(100*np.sqrt(mse_train[-1]),2)
    NMAE_train  = round(100*mae_train[-1],2)
    NRMSE_val = round(100*np.sqrt(mse_val[-1]),2)
    NMAE_val  = round(100*mae_val[-1],2)
    NRMSE_test = round(100*np.sqrt(mse_test[-1]),2)
    NMAE_test  = round(100*mae_test[-1],2)

    if mse_val[-1] == np.min(mse_val):
        torch.save(dmae1.state_dict(), 'DMAE1_'+DATA_NUM+'_'+MISS_NUM+'.pt')

    if epoch == 0 or (epoch+1) % 100 == 0:
        print("epoch: {}".format(epoch+1).ljust(15), end="")
        print("NRMSE_train: {}%".format(NRMSE_train).ljust(25), end="")
        print("NMAE_train: {}%".format(NMAE_train).ljust(25), end="")
        print("NRMSE_val: {}%".format(NRMSE_val).ljust(25), end="")
        print("NMAE_val: {}%".format(NMAE_val).ljust(25), end="")
        print("NRMSE_test: {}%".format(NRMSE_test).ljust(25), end="")
        print("NMAE_test: {}%".format(NMAE_test).ljust(25))

In [ ]:
# Denoising Masked AutoEncoder (Test)

batch_size = 128
total_batch = int(len(data_train_miss)/batch_size) + 1

dmae1 = AED()

mse_train = []; mse_val = []; mse_test = []
mae_train = []; mae_val = []; mae_test = []

dmae1.load_state_dict(torch.load('DMAE1_'+DATA_NUM+'_'+MISS_NUM+'.pt'))

data_train_DMAE1 = dmae1.forward(torch.tensor(data_train_miss2, dtype=torch.float))[1].detach().numpy()
data_val_DMAE1   = dmae1.forward(torch.tensor(data_val_miss2, dtype=torch.float))[1].detach().numpy()
data_test_DMAE1  = dmae1.forward(torch.tensor(data_test_miss, dtype=torch.float))[1].detach().numpy()

data_train_DMAE1[data_train_miss - data_train_miss2 == 0] = 0
data_val_DMAE1[data_val_miss - data_val_miss2 == 0] = 0
data_test_DMAE1[data_test_miss != 0] = 0

mse_train += [np.mean(np.square(data_train_DMAE1 - (data_train_miss - data_train_miss2)))*mean_corrector_train]
mae_train += [np.mean(np.abs(data_train_DMAE1 - (data_train_miss - data_train_miss2)))*mean_corrector_train]
mse_val += [np.mean(np.square(data_val_DMAE1 - (data_val_miss - data_val_miss2)))*mean_corrector_val]
mae_val += [np.mean(np.abs(data_val_DMAE1 - (data_val_miss - data_val_miss2)))*mean_corrector_val]
mse_test += [np.mean(np.square(data_test_DMAE1 - (data_test - data_test_miss)))*mean_corrector_test]
mae_test += [np.mean(np.abs(data_test_DMAE1 - (data_test - data_test_miss)))*mean_corrector_test]

NRMSE_train = round(100*np.sqrt(mse_train[-1]),2)
NMAE_train  = round(100*mae_train[-1],2)
NRMSE_val = round(100*np.sqrt(mse_val[-1]),2)
NMAE_val  = round(100*mae_val[-1],2)
NRMSE_test = round(100*np.sqrt(mse_test[-1]),2)
NMAE_test  = round(100*mae_test[-1],2)

print("NRMSE_train: {}%".format(NRMSE_train).ljust(25), end="")
print("NMAE_train: {}%".format(NMAE_train).ljust(25), end="")
print("NRMSE_val: {}%".format(NRMSE_val).ljust(25), end="")
print("NMAE_val: {}%".format(NMAE_val).ljust(25), end="")
print("NRMSE_test: {}%".format(NRMSE_test).ljust(25), end="")
print("NMAE_test: {}%".format(NMAE_test).ljust(25))

In [ ]:
# Denoising Masked AutoEncoder 2 (Training)

batch_size1 = 128
total_batch1 = int(len(data_train_miss)/batch_size1) + 1
batch_size2 = 32
total_batch2 = int(len(data_val_miss)/batch_size2) + 1

dmae2 = AED()
optimizer = optim.Adam(dmae2.parameters(), lr = 0.001)

mse_train = []; mse_val = []; mse_test = []
mae_train = []; mae_val = []; mae_test = []
for epoch in range(10000):
    for i in range(total_batch1):
        batch_x = torch.tensor(data_train_miss2[batch_size1*i:batch_size1*(i+1),:] ,dtype=torch.float)
        batch_y = torch.tensor(data_train_miss2[batch_size1*i:batch_size1*(i+1),:] ,dtype=torch.float)
        batch = [batch_x, batch_y]
        train_MAE(dmae2, batch, optimizer)

    data_train_DMAE2 = dmae2.forward(torch.tensor(data_train_miss2, dtype=torch.float))[1].detach().numpy()
    data_val_DMAE2   = dmae2.forward(torch.tensor(data_val_miss2, dtype=torch.float))[1].detach().numpy()
    data_test_DMAE2  = dmae2.forward(torch.tensor(data_test_miss, dtype=torch.float))[1].detach().numpy()

    data_train_DMAE2[data_train_miss - data_train_miss2 == 0] = 0
    data_val_DMAE2[data_val_miss - data_val_miss2 == 0] = 0
    data_test_DMAE2[data_test_miss != 0] = 0

    mse_train += [np.mean(np.square(data_train_DMAE2 - (data_train_miss - data_train_miss2)))*mean_corrector_train]
    mae_train += [np.mean(np.abs(data_train_DMAE2 - (data_train_miss - data_train_miss2)))*mean_corrector_train]
    mse_val += [np.mean(np.square(data_val_DMAE2 - (data_val_miss - data_val_miss2)))*mean_corrector_val]
    mae_val += [np.mean(np.abs(data_val_DMAE2 - (data_val_miss - data_val_miss2)))*mean_corrector_val]
    mse_test += [np.mean(np.square(data_test_DMAE2 - (data_test - data_test_miss)))*mean_corrector_test]
    mae_test += [np.mean(np.abs(data_test_DMAE2 - (data_test - data_test_miss)))*mean_corrector_test]

    NRMSE_train = round(100*np.sqrt(mse_train[-1]),2)
    NMAE_train  = round(100*mae_train[-1],2)
    NRMSE_val = round(100*np.sqrt(mse_val[-1]),2)
    NMAE_val  = round(100*mae_val[-1],2)
    NRMSE_test = round(100*np.sqrt(mse_test[-1]),2)
    NMAE_test  = round(100*mae_test[-1],2)

    if mse_val[-1] == np.min(mse_val):
        torch.save(dmae2.state_dict(), 'DMAE2_'+DATA_NUM+'_'+MISS_NUM+'.pt')

    if epoch == 0 or (epoch+1) % 100 == 0:
        print("epoch: {}".format(epoch+1).ljust(15), end="")
        print("NRMSE_train: {}%".format(NRMSE_train).ljust(25), end="")
        print("NMAE_train: {}%".format(NMAE_train).ljust(25), end="")
        print("NRMSE_val: {}%".format(NRMSE_val).ljust(25), end="")
        print("NMAE_val: {}%".format(NMAE_val).ljust(25), end="")
        print("NRMSE_test: {}%".format(NRMSE_test).ljust(25), end="")
        print("NMAE_test: {}%".format(NMAE_test).ljust(25))

In [ ]:
# Denoising Masked AutoEncoder (Test)

batch_size = 128
total_batch = int(len(data_train_miss)/batch_size) + 1

dmae2 = AED()

mse_train = []; mse_val = []; mse_test = []
mae_train = []; mae_val = []; mae_test = []

dmae2.load_state_dict(torch.load('DMAE2_'+DATA_NUM+'_'+MISS_NUM+'.pt'))

data_train_DMAE2 = dmae2.forward(torch.tensor(data_train_miss2, dtype=torch.float))[1].detach().numpy()
data_val_DMAE2   = dmae2.forward(torch.tensor(data_val_miss2, dtype=torch.float))[1].detach().numpy()
data_test_DMAE2  = dmae2.forward(torch.tensor(data_test_miss, dtype=torch.float))[1].detach().numpy()

data_train_DMAE2[data_train_miss - data_train_miss2 == 0] = 0
data_val_DMAE2[data_val_miss - data_val_miss2 == 0] = 0
data_test_DMAE2[data_test_miss != 0] = 0

mse_train += [np.mean(np.square(data_train_DMAE2 - (data_train_miss - data_train_miss2)))*mean_corrector_train]
mae_train += [np.mean(np.abs(data_train_DMAE2 - (data_train_miss - data_train_miss2)))*mean_corrector_train]
mse_val += [np.mean(np.square(data_val_DMAE2 - (data_val_miss - data_val_miss2)))*mean_corrector_val]
mae_val += [np.mean(np.abs(data_val_DMAE2 - (data_val_miss - data_val_miss2)))*mean_corrector_val]
mse_test += [np.mean(np.square(data_test_DMAE2 - (data_test - data_test_miss)))*mean_corrector_test]
mae_test += [np.mean(np.abs(data_test_DMAE2 - (data_test - data_test_miss)))*mean_corrector_test]

NRMSE_train = round(100*np.sqrt(mse_train[-1]),2)
NMAE_train  = round(100*mae_train[-1],2)
NRMSE_val = round(100*np.sqrt(mse_val[-1]),2)
NMAE_val  = round(100*mae_val[-1],2)
NRMSE_test = round(100*np.sqrt(mse_test[-1]),2)
NMAE_test  = round(100*mae_test[-1],2)

print("NRMSE_train: {}%".format(NRMSE_train).ljust(25), end="")
print("NMAE_train: {}%".format(NMAE_train).ljust(25), end="")
print("NRMSE_val: {}%".format(NRMSE_val).ljust(25), end="")
print("NMAE_val: {}%".format(NMAE_val).ljust(25), end="")
print("NRMSE_test: {}%".format(NRMSE_test).ljust(25), end="")
print("NMAE_test: {}%".format(NMAE_test).ljust(25))